In [1]:
# import dependencies
import pandas as pd
import json
import datetime as dt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
# import files and create dataframe
campaigns_df = pd.DataFrame()

for i in range(0,49):
    if i < 10:
        file_location = f'raw_data/Kickstarter00{i}.csv'
    else:
        file_location = f'raw_data/Kickstarter0{i}.csv'
    new_sheet_df = pd.read_csv(file_location)
    dfs = [campaigns_df, new_sheet_df]
    campaigns_df = pd.concat(dfs)
    
campaigns_df = campaigns_df.reset_index()

In [3]:
# extract individual features from json objects and append to dataframe
cat_names = []
cat_ids = []
parent_cat_ids = []
creator_names = []
creator_profiles = []
start_dates = []
deadlines = []
campaign_lengths = []
states = []
    
for i in range(0, len(campaigns_df['launched_at'])):
    cat = json.loads(campaigns_df['category'][i])
    cat_id = cat['id']
    cat_name = cat['name']
    try:
        parent_cat_id = cat['parent_id']
    except KeyError:
        parent_cat_id = '0'
    cat_ids.append(cat_id)
    cat_names.append(cat_name)
    parent_cat_ids.append(parent_cat_id)
    
    creator = json.loads(campaigns_df['creator'][i])
    creator_profile = creator['urls']['web']['user']
    creator_name = creator['name']
    creator_names.append(creator_name)
    creator_profiles.append(creator_profile)
    
    start_date_ts = campaigns_df['launched_at'][i]
    deadline_ts = campaigns_df['deadline'][i]
    campaign_length = round((deadline_ts - start_date_ts)/86400,2)
    start_date = dt.datetime.fromtimestamp(int(start_date_ts)).strftime('%Y-%m-%d %H:%M:%S')
    deadline = dt.datetime.fromtimestamp(int(deadline_ts)).strftime('%Y-%m-%d %H:%M:%S')
    start_dates.append(start_date)
    deadlines.append(deadline)
    campaign_lengths.append(campaign_length)
    
    try:
        state = json.loads(campaigns_df['location'][i])['state']
    except (KeyError, TypeError):
        state = 'na'  
    states.append(state)
    
campaigns_df['category_id'] = cat_ids
campaigns_df['category_name'] = cat_names
campaigns_df['parent_category_id'] = parent_cat_ids
campaigns_df['creator_profile'] = creator_profiles
campaigns_df['creator_name'] = creator_names
campaigns_df['start_date'] = start_dates
campaigns_df['deadline'] = deadlines
campaigns_df['campaign_length'] = campaign_lengths
campaigns_df['state_or_province'] = states

In [4]:
# drop features that are not of interest
features_of_interest = campaigns_df.drop(['category', 'converted_pledged_amount','created_at','creator','currency_symbol',
                                          'currency_trailing_code','current_currency','disable_communication','is_starrable',
                                          'launched_at','photo','profile','source_url','spotlight','staff_pick',
                                          'state_changed_at','static_usd_rate','urls','usd_pledged','usd_type','location',
                                          'friends','is_backing','is_starred','permissions'], axis=1)

features_of_interest.head()

,index,backers_count,blurb,country,currency,deadline,fx_rate,goal,id,name,...,slug,state,category_id,category_name,parent_category_id,creator_profile,creator_name,start_date,campaign_length,state_or_province
0,0,80,I will be an artist-in-residence at Elsewhere ...,US,USD,2012-04-19 15:16:00,1.0,2800.0,1562040083,Elsewhere Studios artist-in-residency program!,...,elsewhere-studios-artist-in-residency-program,successful,54,Mixed Media,1,https://www.kickstarter.com/profile/hilaryemer...,Hilary Emerson Lay,2012-03-28 16:14:20,21.96,CO
1,1,47,We are looking to bring a Visiting Sculptor fr...,US,USD,2012-04-20 17:06:38,1.0,3900.0,1437561817,Martin Luther King Jr. Sculpture on Campus!,...,martin-luther-king-jr-sculpture-on-campus,successful,1,Art,0,https://www.kickstarter.com/profile/1536850207,Csub Arts Humanities Matter,2012-03-28 17:06:38,23.00,CA
2,2,80,Surrealistic oil paintings capturing the metam...,US,USD,2012-04-16 22:59:00,1.0,750.0,574125813,EMERGENCE: Surreal Oil Paintings by J.J. Long,...,emergence-surreal-oil-paintings-by-jj-long,successful,23,Painting,1,https://www.kickstarter.com/profile/2039713907,J.J. Long,2012-03-28 18:01:19,19.21,MA
3,3,82,1000 Artists is a public art-making installati...,US,USD,2012-05-07 20:22:25,1.0,4500.0,858990141,1000 Artists: Presidential Inauguration 2013,...,1000-artists-presidential-inauguration-2013,successful,53,Public Art,1,https://www.kickstarter.com/profile/17146650,Andrew Purchin,2012-03-28 20:22:25,40.00,DC
4,4,31,P.M.A.F.T.W. my upcoming solo show June 2012 a...,US,USD,2012-04-02 21:57:23,1.0,1000.0,566704999,P.M.A.F.T.W.,...,pmaftw,successful,23,Painting,1,https://www.kickstarter.com/profile/1442314751,Ryan Jacob Smith,2012-03-28 21:57:23,5.00,OR


In [5]:
# create dictionary of category names by id
cat_id_list = []
cat_name_list = []

for i in range(0, len(cat_ids)):
    if cat_ids[i] in cat_id_list:
        cat_id_list = cat_id_list
    else:
        cat_id_list.append(cat_ids[i])
        cat_name_list.append(cat_names[i])
        
cat_dic = {}
for i in range(0, len(cat_id_list)):
    cat_dic[cat_id_list[i]] = cat_name_list[i]

# extract parent category names from parent category ids

parent_cat_names = []

for parent_id in parent_cat_ids:
    try:
        parent_cat_name = cat_dic[parent_id]
    except KeyError:
        parent_cat_name = 'none'
    parent_cat_names.append(parent_cat_name)
    
features_of_interest['parent_category'] = parent_cat_names

In [6]:
# perform text analysis (length and sentiment) on the campaign blurb
blurbs = features_of_interest["blurb"]
blurb_lengths = []
blurb_sentiment_compound = []
blurb_sentiment_positive = []
blurb_sentiment_negative = []
blurb_sentiment_neutral = []

for blurb in blurbs:
    try:
        blurb_length = len(blurb)
        sentiment = analyzer.polarity_scores(blurb)
    except TypeError:
        blurb_length = 0
        sentiment = {'compound':0,'pos':0,'neg':0,'neu':1}
    compound = sentiment['compound']
    positive = sentiment['pos']
    negative = sentiment['neg']
    neutral = sentiment['neu']
    blurb_sentiment_compound.append(compound)
    blurb_sentiment_positive.append(positive)
    blurb_sentiment_negative.append(negative)
    blurb_sentiment_neutral.append(neutral)    
    blurb_lengths.append(blurb_length)

features_of_interest['blurb_length'] = blurb_lengths
features_of_interest['blurb_sentiment_compound'] = blurb_sentiment_compound
features_of_interest['blurb_sentiment_positive'] = blurb_sentiment_positive
features_of_interest['blurb_sentiment_negative'] = blurb_sentiment_negative
features_of_interest['blurb_sentiment_neutral'] = blurb_sentiment_neutral

In [7]:
# perform text analysis (length and sentiment) on the campaign title
title_lengths = []
title_sentiment_compound = []
title_sentiment_positive = []
title_sentiment_negative = []
title_sentiment_neutral = []

for title in features_of_interest['name']:
    try:
        title_length = len(title)
        sentiment = analyzer.polarity_scores(title)
    except TypeError:
        title_length = 0
        sentiment = {'compound':0,'pos':0,'neg':0,'neu':1}
    compound = sentiment['compound']
    positive = sentiment['pos']
    negative = sentiment['neg']
    neutral = sentiment['neu']
    title_sentiment_compound.append(compound)
    title_sentiment_positive.append(positive)
    title_sentiment_negative.append(negative)
    title_sentiment_neutral.append(neutral)    
    title_lengths.append(title_length)
    
features_of_interest['title_length'] = title_lengths
features_of_interest['title_sentiment_compound'] = title_sentiment_compound
features_of_interest['title_sentiment_positive'] = title_sentiment_positive
features_of_interest['title_sentiment_negative'] = title_sentiment_negative
features_of_interest['title_sentiment_neutral'] = title_sentiment_neutral

In [8]:
# calculate the campaign goal in USD
usd_goal = features_of_interest['goal']*features_of_interest['fx_rate']
features_of_interest['usd_goal'] = usd_goal

In [9]:
# determine the percentage of funding received
pct_complete = features_of_interest['pledged']/features_of_interest['goal']
features_of_interest['pct_complete'] = pct_complete

In [10]:
# keep only projects that have been completed (success, fail, cancel)
success_df = features_of_interest[features_of_interest.state == 'successful']
failure_df = features_of_interest[features_of_interest.state == 'failed']
canceled_df = features_of_interest[features_of_interest.state == 'canceled']
dfs = [success_df, failure_df, canceled_df]
kickstarter_df = pd.concat(dfs)

live_projects = len(features_of_interest) - len(kickstarter_df)
print(f'{live_projects} campaigns removed from dataset')

kickstarter_df.head()

7477 campaigns removed from dataset


,index,backers_count,blurb,country,currency,deadline,fx_rate,goal,id,name,...,blurb_sentiment_positive,blurb_sentiment_negative,blurb_sentiment_neutral,title_length,title_sentiment_compound,title_sentiment_positive,title_sentiment_negative,title_sentiment_neutral,usd_goal,pct_complete
0,0,80,I will be an artist-in-residence at Elsewhere ...,US,USD,2012-04-19 15:16:00,1.0,2800.0,1562040083,Elsewhere Studios artist-in-residency program!,...,0.000,0.0,1.000,46,0.0,0.0,0.0,1.0,2800.0,1.284286
1,1,47,We are looking to bring a Visiting Sculptor fr...,US,USD,2012-04-20 17:06:38,1.0,3900.0,1437561817,Martin Luther King Jr. Sculpture on Campus!,...,0.084,0.0,0.916,43,0.0,0.0,0.0,1.0,3900.0,1.055641
2,2,80,Surrealistic oil paintings capturing the metam...,US,USD,2012-04-16 22:59:00,1.0,750.0,574125813,EMERGENCE: Surreal Oil Paintings by J.J. Long,...,0.000,0.0,1.000,45,0.0,0.0,0.0,1.0,750.0,4.166667
3,3,82,1000 Artists is a public art-making installati...,US,USD,2012-05-07 20:22:25,1.0,4500.0,858990141,1000 Artists: Presidential Inauguration 2013,...,0.000,0.0,1.000,44,0.0,0.0,0.0,1.0,4500.0,1.019111
4,4,31,P.M.A.F.T.W. my upcoming solo show June 2012 a...,US,USD,2012-04-02 21:57:23,1.0,1000.0,566704999,P.M.A.F.T.W.,...,0.000,0.0,1.000,12,0.0,0.0,0.0,1.0,1000.0,1.036000


In [11]:
# drop indices and set new index
kickstarter_df = kickstarter_df.reset_index()
kickstarter_df = kickstarter_df.drop(['level_0','index'], axis=1)

In [12]:
# review counts for each outcome
kickstarter_df.groupby('state').count()['id']

state
canceled        8823
failed         73634
successful    105680
Name: id, dtype: int64

# Save Data to SQLite DB

In [13]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
import pymysql
pymysql.install_as_MySQLdb()

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [14]:
# create database
connection_string = "sqlite:///kickstarter_campaigns.sqlite"
engine = create_engine(connection_string)
conn = engine.connect()

In [15]:
# create ORM classes for each table
class KickstarterCampaigns(Base):
    __tablename__ = 'kickstarter_campaigns'

    index = Column(Integer, primary_key=True)
    backers_count = Column(Integer)
    blurb = Column(Text)
    country = Column(Text)
    currency = Column(Text)
    deadline = Column(Text)
    fx_rate = Column(Float)
    goal = Column(Float)
    id = Column(Integer)
    name = Column(Text)
    pledged = Column(Float)
    slug = Column(Text)
    state = Column(Text)
    category_id = Column(Integer)
    category_name = Column(Text)
    parent_category_id = Column(Integer)
    creator_profile = Column(Text)
    creator_name = Column(Text)
    start_date = Column(Text)
    campaign_length = Column(Float)
    state_or_province = Column(Text)
    parent_category = Column(Text)
    blurb_length = Column(Integer)
    blurb_sentiment_compound = Column(Float)
    blurb_sentiment_positive = Column(Float)
    blurb_sentiment_negative = Column(Float)
    blurb_sentiment_neutral = Column(Float)
    title_length = Column(Integer)
    title_sentiment_compound = Column(Float)
    title_sentiment_positive = Column(Float)
    title_sentiment_negative = Column(Float)
    title_sentiment_neutral = Column(Float)
    usd_goal = Column(Float)
    pct_complete = Column(Float)
         
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
Base.metadata.create_all(engine)

In [16]:
# write data to tables
data_to_write = kickstarter_df.to_dict(orient='records')
metadata = MetaData(bind=engine)
metadata.reflect()

kickstarter_table = sqlalchemy.Table('kickstarter_campaigns', metadata, autoload=True)

conn.execute(kickstarter_table.insert(), data_to_write)

In [17]:
# confirm data properly written to table

kickstarter_campaigns_head = conn.execute("select * from kickstarter_campaigns limit 5").fetchall()

print("First 5 rows of campaigns:")
for i in range(0,5):
    print(kickstarter_campaigns_head[i])

First 5 rows of campaigns:
(1, 80, 'I will be an artist-in-residence at Elsewhere Studios this summer, where I will stretch my wings and see where I land!', 'US', 'USD', '2012-04-19 15:16:00', 1.0, 2800.0, 1562040083, 'Elsewhere Studios artist-in-residency program!', 3596.0, 'elsewhere-studios-artist-in-residency-program', 'successful', 54, 'Mixed Media', 1, 'https://www.kickstarter.com/profile/hilaryemersonlay', 'Hilary Emerson Lay', '2012-03-28 16:14:20', 21.96, 'CO', 'Art', 118, 0.0, 0.0, 0.0, 1.0, 46, 0.0, 0.0, 0.0, 1.0, 2800.0, 1.2842857142857143)
(2, 47, 'We are looking to bring a Visiting Sculptor from the Ivory Coast to work with our students to create a sculpture of MLK for our campus.', 'US', 'USD', '2012-04-20 17:06:38', 1.0, 3900.0, 1437561817, 'Martin Luther King Jr. Sculpture on Campus!', 4117.0, 'martin-luther-king-jr-sculpture-on-campus', 'successful', 1, 'Art', 0, 'https://www.kickstarter.com/profile/1536850207', 'Csub Arts Humanities Matter', '2012-03-28 17:06:38', 23